In [2]:
import tensorflow as tf
import os
import requests
import matplotlib.pyplot as plt
from __future__ import print_function
import random
from sklearn.manifold import TSNE
import zipfile
import numpy as np

In [3]:
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data    

In [4]:
path = os.getcwd() + '/text8.zip'
read_file = read_data(path)

In [5]:
print(len(read_file))

17005207


In [70]:
read_file

['anarchism',
 'originated',
 'as',
 'a',
 'term',
 'of',
 'abuse',
 'first',
 'used',
 'against',
 'early',
 'working',
 'class',
 'radicals',
 'including',
 'the',
 'diggers',
 'of',
 'the',
 'english',
 'revolution',
 'and',
 'the',
 'sans',
 'culottes',
 'of',
 'the',
 'french',
 'revolution',
 'whilst',
 'the',
 'term',
 'is',
 'still',
 'used',
 'in',
 'a',
 'pejorative',
 'way',
 'to',
 'describe',
 'any',
 'act',
 'that',
 'used',
 'violent',
 'means',
 'to',
 'destroy',
 'the',
 'organization',
 'of',
 'society',
 'it',
 'has',
 'also',
 'been',
 'taken',
 'up',
 'as',
 'a',
 'positive',
 'label',
 'by',
 'self',
 'defined',
 'anarchists',
 'the',
 'word',
 'anarchism',
 'is',
 'derived',
 'from',
 'the',
 'greek',
 'without',
 'archons',
 'ruler',
 'chief',
 'king',
 'anarchism',
 'as',
 'a',
 'political',
 'philosophy',
 'is',
 'the',
 'belief',
 'that',
 'rulers',
 'are',
 'unnecessary',
 'and',
 'should',
 'be',
 'abolished',
 'although',
 'there',
 'are',
 'differing',
 '

In [7]:
vocabulary_size = 50000
import collections

In [8]:
def build_dataset(words):
    count = [['UNK',-1]]
    
    count.extend(collections.Counter(words).most_common(vocabulary_size-1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0
            unk_count = unk_count + 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    
    return data,count,dictionary,reverse_dictionary

data,count,dictionary,reverse_dictionary = build_dataset(read_file) 


In [30]:
data_index = 0

In [89]:

def generate_batch_skip_gram(batch_size, window_size):
  # data_index is updated by 1 everytime we read a data point
    global data_index 
    
  # two numpy arras to hold target words (batch)
  # and context words (labels)
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    
  # span defines the total window size, where
  # data we consider at an instance looks as follows. 
  # [ skip_window target skip_window ]
    span = 2 * window_size + 1 
    
  # The buffer holds the data contained within the span
    buffer = collections.deque(maxlen=span)   #deque are list like containers with fast append from both the ends
  
  # Fill the buffer and update the data_index
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
  
  # This is the number of context words we sample for a single target word
    num_samples = 2*window_size 

  # We break the batch reading into two for loops
  # The inner for loop fills in the batch and labels with 
  # num_samples data points using data contained withing the span
  # The outper for loop repeat this for batch_size//num_samples times
  # to produce a full batch
    for i in range(batch_size // num_samples):
        k=0
    # avoid the target word itself as a prediction
    # fill in batch and label numpy arrays
        for j in list(range(window_size))+list(range(window_size+1,2*window_size+1)):
            batch[i * num_samples + k] = buffer[window_size]
            labels[i * num_samples + k, 0] = buffer[j]
            k += 1 
    
    # Everytime we read num_samples data points,
    # we have created the maximum number of datapoints possible
    # withing a single span, so we need to move the span by 1
    # to create a fresh new span
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
    return batch, labels

print('data:', [reverse_dictionary[di] for di in data[:40]])

for window_size in [1, 2]:
    data_index = 0
    batch, labels = generate_batch_skip_gram(batch_size=8, window_size=1)
    print('\nwith window_size = %d:' %window_size)
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])

data: ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'UNK', 'of', 'the', 'french', 'revolution', 'whilst', 'the', 'term', 'is', 'still', 'used', 'in', 'a', 'pejorative', 'way', 'to']

with window_size = 1:
    batch: ['originated', 'originated', 'originated', 'originated', 'originated', 'originated', 'originated', 'originated']
    labels: ['anarchism', 'as', 'anarchism', 'as', 'anarchism', 'as', 'anarchism', 'as']

with window_size = 2:
    batch: ['originated', 'originated', 'originated', 'originated', 'originated', 'originated', 'originated', 'originated']
    labels: ['anarchism', 'as', 'anarchism', 'as', 'anarchism', 'as', 'anarchism', 'as']


In [31]:
data_index = (data_index + 1) % len(data)

In [56]:
data_index = (data_index + 1) % len(data)
data_index = (data_index + 1) % len(data)
data_index

21

In [57]:
data_index = 0

In [58]:
for _ in range(9):
        
        data_index = (data_index + 1) % len(data)
        print(data_index)

1
2
3
4
5
6
7
8
9


In [60]:
window_size = 4
print(list(range(window_size)))
print(list(range(window_size+1,2*window_size+1)))

[0, 1, 2, 3]
[5, 6, 7, 8]
